# Example data anonymization

In Pega CDH 8.5 and up, it's now possible to record the historical data as seen by the Adaptive Models. See [this academy challenge](https://academy.pega.com/challenge/exporting-historical-data/v4) for reference. This historical data can be further used to experiment with offline models, but also to fine-tune the OOTB Gradient Boosting model. However, sharing this information with Pega can be sensitive as it contains raw predictor data. 

To this end, we provide a simple and transparent script to fully anonimize this dataset.

The DataAnonymization script is now part of pdstools, and you can import it directly as such.

In [1]:
# These lines are only for rendering in the docs, and are hidden through Jupyter tags
# Do not run if you're running the notebook seperately

import os  
import sys
import plotly.io as pio
pio.renderers.default = "notebook_connected"

sys.path.append("../../../")
sys.path.append('../../python')

In [2]:
from pdstools import ADMDatamart
from pdstools import Config, DataAnonymization
import polars as pl

## Input data

To demonstrate this process, we're going to anonymise this toy example dataframe:

In [3]:
pl.read_ndjson('../../../../data/SampleHDS.json')

Context_Name,Customer_MaritalStatus,Customer_CLV,Customer_City,IH_Web_Inbound_Accepted_pxLastGroupID,Decision_Outcome
str,str,i64,str,str,str
"""FirstMortgage3…","""Married""",1460,"""Port Raoul""","""Account""","""Rejected"""
"""FirstMortgage3…","""Unknown""",669,"""Laurianneshire…","""AutoLoans""","""Accepted"""
"""MoneyMarketSav…","""No Resp+""",1174,"""Jacobshaven""","""Account""","""Rejected"""
"""BasicChecking""","""Unknown""",1476,"""Lindton""","""Account""","""Rejected"""
"""BasicChecking""","""Married""",1211,"""South Jimmiesh…","""DepositAccount…","""Accepted"""
"""UPlusFinPerson…","""No Resp+""",533,"""Bergeville""",null,"""Rejected"""
"""BasicChecking""","""No Resp+""",555,"""Willyville""","""DepositAccount…","""Rejected"""


As you can see, this dataset consists of regular predictors, IH predictors, context keys and the outcome column. Additionally, some columns are numeric, others are strings. Let's first initialize the DataAnonymization class.

In [4]:
anon = DataAnonymization(hds_folder='../../../../data/')

By default, the class applies a set of anonymisation techniques:
- Column names are remapped to a non-descriptive name
- Categorical values are hashed with a random seed
- Numerical values are normalized between 0 and 1
- Outcomes are mapped to a binary outcome.

To apply these techniques, simply call `.process()`:

In [5]:
anon.process()

PREDICTOR_0,filename,PREDICTOR_2,PREDICTOR_3,Context_Name,IH_PREDICTOR_0,Decision_Outcome
str,str,f64,str,str,str,bool
"""17539876647055…","""../../../../da…",1.2927e19,"""41349085870085…","""80987082883961…","""95879245320162…",false
"""15348054024215…","""../../../../da…",1.4856e19,"""18168726380772…","""80987082883961…","""13838880526453…",true
"""17672253662705…","""../../../../da…",5.6458e17,"""17927906522830…","""46450833782493…","""95879245320162…",false
"""12321517161363…","""../../../../da…",4.0723e18,"""18168726380772…","""11573324638857…","""95879245320162…",false
"""37917372765283…","""../../../../da…",1.4677e19,"""41349085870085…","""11573324638857…","""91213303555248…",true
"""12480785833169…","""../../../../da…",1.6330e19,"""17927906522830…","""15274885383750…",null,false
"""32585232254503…","""../../../../da…",8.7675e18,"""17927906522830…","""11573324638857…","""91213303555248…",false


To trace back the columns to their original names, the class also contains a mapping, which does not have to be provided.

In [6]:
anon.column_mapping

{'Customer_City': 'PREDICTOR_0',
 'filename': 'filename',
 'Customer_CLV': 'PREDICTOR_2',
 'Customer_MaritalStatus': 'PREDICTOR_3',
 'Context_Name': 'Context_Name',
 'IH_Web_Inbound_Accepted_pxLastGroupID': 'IH_PREDICTOR_0',
 'Decision_Outcome': 'Decision_Outcome'}

## Configs

Each capability can optionally be turned off - see below for the full list of config options, and refer to the API reference for the full description.

In [7]:
dict(zip(Config.__init__.__code__.co_varnames[1:], Config.__init__.__defaults__))

{'config_file': None,
 'hds_folder': '.',
 'use_datamart': False,
 'datamart_folder': 'datamart',
 'output_format': 'ndjson',
 'output_folder': 'output',
 'mapping_file': 'mapping.map',
 'mask_predictor_names': True,
 'mask_context_key_names': False,
 'mask_ih_names': True,
 'mask_outcome_name': False,
 'mask_predictor_values': True,
 'mask_context_key_values': True,
 'mask_ih_values': True,
 'mask_outcome_values': True,
 'context_key_label': 'Context_*',
 'ih_label': 'IH_*',
 'outcome_column': 'Decision_Outcome',
 'positive_outcomes': ['Accepted', 'Clicked'],
 'negative_outcomes': ['Rejected', 'Impression'],
 'special_predictors': ['Decision_DecisionTime',
  'Decision_OutcomeTime',
  'Decision_Rank'],
 'sample_percentage_schema_inferencing': 0.01}

It's easy to change these parameters by just passing the keyword arguments. In the following example, we
- Keep the IH predictor names
- Keep the outcome values
- Keep the context key values
- Keep the context key predictor names

In [8]:
anon = DataAnonymization(
    hds_folder="../../../../data/",
    mask_ih_names=False,
    mask_outcome_values=False,
    mask_context_key_values=False,
    mask_context_key_names=False,
)
anon.process()


PREDICTOR_0,filename,PREDICTOR_2,PREDICTOR_3,Context_Name,IH_Web_Inbound_Accepted_pxLastGroupID,Decision_Outcome
str,str,f64,str,str,str,str
"""14851886842564…","""../../../../da…",1.2927e19,"""44926524940448…","""FirstMortgage3…","""16688242752767…","""Rejected"""
"""14997546074769…","""../../../../da…",1.4856e19,"""32365032243017…","""FirstMortgage3…","""99764767432540…","""Accepted"""
"""12120000286308…","""../../../../da…",5.6458e17,"""89529192846172…","""MoneyMarketSav…","""16688242752767…","""Rejected"""
"""97931351986680…","""../../../../da…",4.0723e18,"""32365032243017…","""BasicChecking""","""16688242752767…","""Rejected"""
"""11422333981924…","""../../../../da…",1.4677e19,"""44926524940448…","""BasicChecking""","""62416336811533…","""Accepted"""
"""17239680823090…","""../../../../da…",1.6330e19,"""89529192846172…","""UPlusFinPerson…",null,"""Rejected"""
"""14193101593387…","""../../../../da…",8.7675e18,"""89529192846172…","""BasicChecking""","""62416336811533…","""Rejected"""


The configs can also be written and read as such:

In [9]:
anon.config.save_to_config_file('config.json')

In [10]:
anon = DataAnonymization(config=Config(config_file='config.json'))
anon.process()

PREDICTOR_0,filename,PREDICTOR_2,PREDICTOR_3,Context_Name,IH_Web_Inbound_Accepted_pxLastGroupID,Decision_Outcome
str,str,f64,str,str,str,str
"""10283353066470…","""../../../../da…",1.2927e19,"""18084776691296…","""FirstMortgage3…","""17208769258503…","""Rejected"""
"""11586048747768…","""../../../../da…",1.4856e19,"""40587421225858…","""FirstMortgage3…","""37076463322729…","""Accepted"""
"""37501303279044…","""../../../../da…",5.6458e17,"""16069169145550…","""MoneyMarketSav…","""17208769258503…","""Rejected"""
"""17635983049496…","""../../../../da…",4.0723e18,"""40587421225858…","""BasicChecking""","""17208769258503…","""Rejected"""
"""61148340539992…","""../../../../da…",1.4677e19,"""18084776691296…","""BasicChecking""","""88152969900667…","""Accepted"""
"""13738456347826…","""../../../../da…",1.6330e19,"""16069169145550…","""UPlusFinPerson…",null,"""Rejected"""
"""90910864332012…","""../../../../da…",8.7675e18,"""16069169145550…","""BasicChecking""","""88152969900667…","""Rejected"""


## Exporting
Two functions export:
- `create_mapping_file()` writes the mapping file of the predictor names
- `write_to_output()` writes the processed dataframe to disk

Write to output accepts the following extensions: `["ndjson", "parquet", "arrow", "csv"]`

In [11]:
anon.create_mapping_file()
with open('mapping.map') as f:
    print(f.read())

Customer_City=PREDICTOR_0
filename=filename
Customer_CLV=PREDICTOR_2
Customer_MaritalStatus=PREDICTOR_3
Context_Name=Context_Name
IH_Web_Inbound_Accepted_pxLastGroupID=IH_Web_Inbound_Accepted_pxLastGroupID
Decision_Outcome=Decision_Outcome



In [12]:
anon.write_to_output(ext='arrow')

In [13]:
pl.read_ipc('output/hds.arrow')

PREDICTOR_0,PREDICTOR_2,PREDICTOR_3,Context_Name,IH_Web_Inbound_Accepted_pxLastGroupID,Decision_Outcome
str,f64,str,str,str,str
"""17038248624602…",1.2927e19,"""20694492732625…","""FirstMortgage3…","""15321099501317…","""Rejected"""
"""24215741916863…",1.4856e19,"""84868999332691…","""FirstMortgage3…","""79749036396415…","""Accepted"""
"""17200951222116…",5.6458e17,"""64283176547630…","""MoneyMarketSav…","""15321099501317…","""Rejected"""
"""11119925957160…",4.0723e18,"""84868999332691…","""BasicChecking""","""15321099501317…","""Rejected"""
"""14269468836825…",1.4677e19,"""20694492732625…","""BasicChecking""","""15223105078709…","""Accepted"""
"""86046302705351…",1.6330e19,"""64283176547630…","""UPlusFinPerson…",null,"""Rejected"""
"""13399525151303…",8.7675e18,"""64283176547630…","""BasicChecking""","""15223105078709…","""Rejected"""


## Advanced: Hash fuctions

By default, we use [the same hashing algorithm Polars](https://pola-rs.github.io/polars/py-polars/html/reference/expressions/api/polars.Expr.hash.html#polars.Expr.hash) uses: [xxhash](https://github.com/Cyan4973/xxHash), as implemented [here](https://github.com/pola-rs/polars/blob/3f287f370b3c388ed2f3f218b2c096382548136f/polars/polars-core/src/vector_hasher.rs#L266). xxhash is fast to compute, and you can check its performance in collision, dispersion and randomness [here](https://github.com/Cyan4973/xxHash/tree/dev/tests). 

xxhash accepts four distinct seeds, but by default we set the seeds to `0`. It is possible to set the `seed` argument of the `process()` function to `'random'`, which will set all four seeds to a random integer between `0` and `1000000000`. Alternatively, it is possible to supply the four seeds manually with arguments `seed`, `seed_1`, `seed_2` and `seed_3`. 

If the xxhash with (random) seed(s) is not deemed sufficiently secure, it is possible to use your own hashing algorithm.

Note that since we're now running python code and not native Polars code anymore, this will be _significantly_ slower. Nonetheless, it is possible.

Just as an example - this is how one would use sha3_256:

In [14]:
from hashlib import sha3_256

anon.process(algorithm=lambda x: sha3_256(x.encode()).hexdigest())

ComputeError: AttributeError: 'int' object has no attribute 'encode'